In [1]:
sample_rate = 16000 # sample rate of the audio
n_way= 3 # number of classes per episode
n_support = 5 # number of support examples per class
n_query = 20 # number of samples per class to use as query
n_train_episodes = int(1000) # number of episodes to generate for training
n_val_episodes = 50 # number of episodes to generate for validation
num_workers = 10 # number of workers to use for data loading
     

In [2]:
import torch
import numpy as np
from torch import nn
import pytorch_lightning as pl
from torchmetrics import Accuracy

from music_fsl.backbone import Backbone
from music_fsl.protonet import PrototypicalNet
     

In [3]:
import torch
from typing import List, Dict, Any

class ClassConditionalDataset(torch.utils.data.Dataset):

    def __getitem__(self, index: int) -> Dict[Any, Any]:
        """
        Grab an item from the dataset. The item returned must be a dictionary. 
        """
        raise NotImplementedError
    
    @property
    def classlist(self) -> List[str]:
        """
        The classlist property returns a list of class labels available in the dataset.
        This property enables users of the dataset to easily access a list of all the classes in the dataset.

        Returns:
            List[str]: A list of class labels available in the dataset. 
        """
        raise NotImplementedError

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        """
        Returns a dictionary where the keys are class labels and the values are 
        lists of indices in the dataset that belong to that class. 
        This property enables users of the dataset to easily access 
        examples that belong to specific classes. 

        Implement me!

        Returns:
            Dict[str, List[int]]: A dictionary mapping class labels to lists of dataset indices. 
        """TutImplementedError

In [4]:
from collections import defaultdict
import mirdata
import music_fsl.util as util
from typing import List, Dict
import os
import sys


class IRMAS(ClassConditionalDataset):
    """
    Initialize a `IRMAS` dataset instance.

    Args:
        instruments (List[str]): A list of instruments to include in the dataset.
        duration (float): The duration of each audio clip in the dataset (in seconds).
        sample_rate (int): The sample rate of the audio clips in the dataset (in Hz).
    """

    INSTRUMENTS = [
        'Cello', 'Clarinet', 'Flute', 'Acoustic guitar',
        'Electric guitar', 'Organ', 'Piano', 
        'Saxophone', 'Trumpet', 'Violin', 'Human singing voice'
    ]

    INSTRUMENTS_KEY = {
        'cel': 'Cello', 'cla' : 'Clarinet', 'flu' : 'Flute', 'gac' : 'Acoustic guitar',
        'gel' : 'Electric guitar', 'org' : 'Organ', 'pia' : 'Piano', 
        'sax' : 'Saxophone', 'tru' : 'Trumpet', 'vio' : 'Violin', 'voi' : 'Human singing voice'
    }
    def __init__(self, 
            instruments: List[str] = None,
            instruments_key: Dict[str, str] = None,
            duration: float = 1.0, 
            sample_rate: int = 16000,
            dataset_path: str = 'irmas',
            val = False,
        ):
        if instruments_key is None:
            instruments_key = self.INSTRUMENTS_KEY

        if instruments is None:
            instruments = self.INSTRUMENTS

        if val == False and dataset_path == 'irmas':
            dir_path = os.path.dirname(os.path.realpath(__file__))
            dataset_path = os.path.join(os.path.dirname(dir_path), dataset_path)
        

        self.instruments = instruments
        self.instruments_key = instruments_key
        self.duration = duration
        self.sample_rate = sample_rate

        # initialize IRMAS path 
        if val == False:
          if os.path.exists(dataset_path):
            self.dataset_path = dataset_path
          else:
            sys.exit("Dataset path does not exist")
        
        if val:
          self.dataset_path = dataset_path

        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.INSTRUMENTS, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        if val:c


    def load_all_tracks(self):
        self.tracks = []
        for subdir, dirs, files in os.walk(self.dataset_path):
            for dir in dirs:
                if dir in self.instruments_key.keys():
                    for subdir_dir, dirs_dir, files_dir in os.walk(os.path.join(subdir, dir)):
                        for file in files_dir:
                            self.tracks.append([os.path.join(subdir, dir, file), self.instruments_key[dir]])

    def load_all_tracks_val(self):
        self.tracks = []
        for paths in self.dataset_path:
          for subdir, dirs, files in os.walk(paths, topdown=True):
              for file in sorted(files)[::2]:
                  f = open(os.path.join(subdir, file), "rb")
                  tags = f.readlines()
                  for tag in tags:
                      tag = tag[:3].decode("utf-8") 
                      f.close()
                      if tag in self.instruments_key.keys():
                          wav_file = os.path.join(subdir, file).replace('.txt', '.wav')
                          if os.path.exists(wav_file):
                            self.tracks.append([wav_file, self.instruments_key[tag]])

    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

   """
`train.py` - Script for training models on different datasets.

This script provides a command-line interface for training models on various datasets.
It currently supports datasets such as IRMAS (Instrument Recognition in Musical Audio) and others.

Usage:
    python train.py <dataset_type> <config_path> <input_filepath> <model_output_filepath>

Arguments:
    dataset_type (str): The type of dataset to train on. Supported values: 'irmas', 'good_sounds', 'wavefiles', 'tiny_sol'.
    config_path (str): Path to the YAML configuration file containing training parameters.
    input_filepath (str): Path to the input data directory.
    model_output_filepath (str): Path to the output directory for saving the trained models.

Example:
    python train.py irmas path/to/config.yaml path/to/dataset irmas_models/

Note:
    This script uses a YAML configuration file to set up training parameters.
    Ensure that the specified dataset type matches the available cases in the script.
"""
    def __len__(self) -> int:
        return len(self.tracks)
        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.INSTRUMENTS, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        if val:
            self.load_all_tracks_val()
        else:
            self.load_all_tracks()


    def load_all_tracks(self):
        self.tracks = []
        for subdir, dirs, files in os.walk(self.dataset_path):
            for dir in dirs:
                if dir in self.instruments_key.keys():
                    for subdir_dir, dirs_dir, files_dir in os.walk(os.path.join(subdir, dir)):
                        for file in files_dir:
                            self.tracks.append([os.path.join(subdir, dir, file), self.instruments_key[dir]])

    def load_all_tracks_val(self):
        self.tracks = []
        for paths in self.dataset_path:
          for subdir, dirs, files in os.walk(paths, topdown=True):
              for file in sorted(files)[::2]:
                  f = open(os.path.join(subdir, file), "rb")
                  tags = f.readlines()
                  for tag in tags:
                      tag = tag[:3].decode("utf-8") 
                      f.close()
                      if tag in self.instruments_key.keys():
                          wav_file = os.path.join(subdir, file).replace('.txt', '.wav')
                          if os.path.exists(wav_file):
                            self.tracks.append([wav_file, self.instruments_key[tag]])

    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

        return data

    def __len__(self) -> int:
        return len(self.tracks)

In [5]:
import random
import torch
from music_fsl.data import ClassConditionalDataset
import music_fsl.util as util

from typing import Tuple, Dict
class EpisodeDataset(torch.utils.data.Dataset):
    """
        A dataset for sampling few-shot learning tasks from a class-conditional dataset.

    Args:
        dataset (ClassConditionalDataset): The dataset to sample episodes from.
        n_way (int): The number of classes to sample per episode.
            Default: 5.
        n_support (int): The number of samples per class to use as support.
            Default: 5.
        n_query (int): The number of samples per class to use as query.
            Default: 20.
        n_episodes (int): The number of episodes to generate.
            Default: 100.
    """
    def __init__(self,
        dataset: ClassConditionalDataset, 
        n_way: int = 5, 
        n_support: int = 5,
        n_query: int = 20,
        n_episodes: int = 100,
    ):
        self.dataset = dataset

        self.n_way = n_way
        self.n_support = n_support
        self.n_query = n_query
        self.n_episodes = n_episodes
    
    def __geUnused PrototypicalNet imported from music_fsl.protonetPylintW0611:unused-import

        Each episode is a tuple of two dictionaries: a support set and a query set.
        The support set contains a set of samples from each of the classes in the
        episode, and the query set contains another set of samples from each of the
        classes. The class labels are added to each item in the support and query
        sets, and the list of classes is also included in each dictionary.

        Yields:
            Tuple[Dict[str, Any], Dict[str, Any]]: A tuple containing the support
            set and the query set for an episode.
        """
        # seed the random number generator so we can reproduce this episode
        rng = random.Random(index)

        # sample the list of classes for this episode
        episode_classlist = rng.sample(self.dataset.classlist, self.n_way)

        # sample the support and query sets for this episode
        support, query = [], []
        for c in episode_classlist:
            # grab the dataset indices for this class
            all_indices = self.dataset.class_to_indices[c]
            # sample the support and query sets for this class
            indices = rng.sample(all_indices, self.n_support + self.n_query)
            items = [self.dataset[i] for i in indices]

            # add the class label to each item
            for item in items:
                item["target"] = torch.tensor(episode_classlist.index(c))

            # split the support and query sets
            support.extend(items[:self.n_support])
            query.extend(items[self.n_support:])

        # collate the support and query sets
        support = util.collate_list_of_dicts(support)
        query = util.collate_list_of_dicts(query)

        support["classlist"] = episode_classlist
        query["classlist"] = episode_classlist
        
        return support, query

    def __len__(self):
        return self.n_episodes

    def print_episode(self, support, query):
        """Print a summary of the support and query sets for an episode.

        Args:
            support (Dict[str, Any]): The support set for an episode.
            query (Dict[str, Any]): The query set for an episode.
        """
        print("Support Set:")
        print(f"  Classlist: {support['classlist']}")
        print(f"  Audio Shape: {support['audio'].shape}")
        print(f"  Target Shape: {support['target'].shape}")
        print()
        print("Query Set:")
        print(f"  Classlist: {query['classlist']}")
        print(f"  Audio Shape: {query['audio'].shape}")
        print(f"  Target Shape: {query['target'].shape}")


In [6]:
TRAIN_INSTRUMENTS = [
        'Flute', 'Organ', 'Saxophone',
        'Trumpet', 'Violin', 'Electric guitar'
    ]

TRAIN_INSTRUMENTS_KEY = {
        'flu' : 'Flute', 'org' : 'Organ',
        'sax' : 'Saxophone', 'tru' : 'Trumpet', 'vio' : 'Violin', 'gel' : 'Electric guitar'
    }

TEST_INSTRUMENTS = [
        'Cello', 'Piano', 'Clarinet', 'Acoustic guitar', 'Human singing voice'
    ]

TEST_INSTRUMENTS_KEY = {
        'cel': 'Cello', 'gac' : 'Acoustic guitar', 'pia' : 'Piano',  'voi' : 'Human singing voice', 'cla' : 'Clarinet'
    }

In [7]:
! ls ../data/irmas/IRMAS-TrainingData

ls: cannot access '../data/irmas/IRMAS-TrainingData': No such file or directory


In [8]:
data_path = "../data/interim/irmas/IRMAS-TrainingData"
train_data = IRMAS(
    instruments=TRAIN_INSTRUMENTS, 
    instruments_key = TRAIN_INSTRUMENTS_KEY,
    sample_rate=sample_rate,
    dataset_path = data_path
)

val_data = IRMAS(
    instruments=TEST_INSTRUMENTS, 
    instruments_key = TEST_INSTRUMENTS_KEY,
    sample_rate=sample_rate,
    dataset_path = data_path
)

In [9]:

print(len(train_data.tracks))
for instrument, indices in train_data.class_to_indices.items():
    print(f"{instrument} has {len(indices)} examples")
print(len(val_data.tracks))
for instrument, indices in val_data.class_to_indices.items():
    print(f"{instrument} has {len(indices)} examples")

3676
Flute has 451 examples
Violin has 580 examples
Saxophone has 626 examples
Trumpet has 577 examples
Organ has 682 examples
Electric guitar has 760 examples
3029
Cello has 388 examples
Piano has 721 examples
Acoustic guitar has 637 examples
Clarinet has 505 examples
Human singing voice has 778 examples


In [10]:
train_episodes = EpisodeDataset(
    dataset=train_data, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_train_episodes
)

val_episodes = EpisodeDataset(
    dataset=val_data, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_val_episodes
)

In [11]:
from torch.utils.data import DataLoader
train_loader = DataLoader(
    train_episodes, 
    batch_size=None,
    num_workers=num_workers
)

val_loader = DataLoader(
    val_episodes, 
    batch_size=None,
    num_workers=num_workers
)

In [12]:
backbone = Backbone(sample_rate=sample_rate)
protonet = PrototypicalNet(backbone)


In [13]:
class FewShotLearner(pl.LightningModule):

    def __init__(self, 
        protonet: nn.Module, 
        learning_rate: float = 1e-3,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.protonet = protonet
        self.learning_rate = learning_rate

        self.loss = nn.CrossEntropyLoss()
        self.metrics = nn.ModuleDict({
            'accuracy': Accuracy()
        })

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def step(self, batch, batch_idx, tag: str):
        support, query = batch

        logits = self.protonet(support, query)
        loss = self.loss(logits, query["target"])

        output = {"loss": loss}
        for k, metric in self.metrics.items():
            output[k] = metric(logits, query["target"])

        for k, v in output.items():
            self.log(f"{k}/{tag}", v)
        return output

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "test")

In [14]:
learner = FewShotLearner(protonet)


/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'protonet' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['protonet'])`.


In [15]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler

trainer = pl.Trainer(
    max_epochs=1,
    log_every_n_steps=1, 
    val_check_interval=50,
    profiler=SimpleProfiler(
        filename="profile.txt",
    ), 
    logger=TensorBoardLogger(
        save_dir=".",
        name="logs"
    ), 
)

# train!
trainer.fit(learner, train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | protonet | PrototypicalNet  | 521 K 
1 | loss     | CrossEntropyLoss | 0     
2 | metrics  | ModuleDict       | 0     
----------------------------------------------
521 K     Trainable params
0         Non-trainable params
521 K     Total params
2.086     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virt

Training: |          | 0/? [00:00<?, ?it/s]

/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virt

...

/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virt

Validation: |          | 0/? [00:00<?, ?it/s]

/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virt

Validation: |          | 0/? [00:00<?, ?it/s]

/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virtualenvs/src-L3lQJWfi-py3.11/lib/python3.11/site-packages/music_fsl/util.py:27: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_duration = librosa.get_duration(filename=audio_path)
/home/agorski/.cache/pypoetry/virt